In [1]:
from syllablecounter import load_model

In [2]:
counter = load_model()
counter

SyllableCounter(
  (embed): Embedding(28, 50)
  (embed_drop): LockedDropout(p=0.1)
  (rnn): GRU(50, 256, bidirectional=True)
  (rnn_bn): TBatchNorm(
    (bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (rnn_drop): LockedDropout(p=0.1)
  (lin1): Linear(in_features=512, out_features=512, bias=True)
  (lin1_bn): TBatchNorm(
    (bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (lin_drop): LockedDropout(p=0.1)
  (lin2): Linear(in_features=512, out_features=256, bias=True)
  (lin2_bn): TBatchNorm(
    (bn): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (out): Linear(in_features=256, out_features=1, bias=True)
)

In [10]:
counter.predict('thx', show_confidence = True)

thanks


(1, 0.9742)